In [28]:
!pip install netcdf4

     |████████████████████████████████| 4.1MB 1.9MB/s eta 0:00:01
     |████████████████████████████████| 307kB 2.3MB/s eta 0:00:01


In [13]:
!pip install numpy

In [29]:
import keras
from keras.layers import LSTM
import xarray as xr
import glob
import numpy as np

In [30]:
# Hyperparameters useful stuff - 
num_epochs = 40
batch_size = 32
#learning_rate = 0.001

DATA_PATH = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/'
MODEL_STORE_PATH = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/storedModel/'

In [31]:
def load_data(start = "2012-01-01", stop = "2019-01-01", season = "SON", 
              era_path = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/'):
    
    era_path = '/home/hanna/lagrings/era_interim_data/'
    """ Want all year put season to. """
    print(glob.glob(era_path + "*_q_*" + season +".nc"))
    q   = xr.open_dataset(glob.glob(era_path + "*_q_*" + season +".nc")[0]).q.values
    r   = xr.open_dataset(glob.glob(era_path + "*_r_*"+ season +".nc")[0]).r.values
    tcc = xr.open_dataset(glob.glob(era_path + "*tcc*"+ season +".nc")[0]).tcc.values
    sp  = xr.open_dataset(glob.glob(era_path + "*sp*"+ season +".nc")[0]).sp.values
    t2m = xr.open_dataset(glob.glob(era_path + "*t2m*"+ season +".nc")[0]).t2m.values
    assert np.shape(q) == np.shape(r) == np.shape(tcc) == np.shape(sp) == np.shape(t2m)
    
    nbr_times, nbr_lats, nbr_lon = np.shape(q)
    
    train = []
    true  = tcc[:, np.newaxis] # is wrong now
    
    
    for i in range(nbr_times):
        one_timestep = np.array([q[i], r[i], sp[i], t2m[i] ])
        train.append(one_timestep)

    return np.array(train), true

In [42]:
train.mean(axis = 0).mean(axis=1).mean(axis=1)

array([8.1528546e-03, 7.2762016e+01, 9.8931422e+04, 2.8904562e+02],
      dtype=float32)

In [44]:
train.shape

(2184, 4, 35, 60)

In [45]:
def train_test_split(data, test_split = 0.2):
    assert test_split < 1, 'test split is given as a decimal number, choose a number between 0, 1'
    indx = int(len(data)*(1-test_split))
    # returns train test
    return data[:indx], data[indx:]

In [80]:
def batch_normalize(data): 
    samples, metvars, lats, lons = data.shape
    #.mean(axis = 0).mean(axis=1).mean(axis=1)
    
    normalized = np.zeros((samples, metvars, lats, lons))
    means   = np.zeros(metvars)
    storage = np.zeros(metvars)
    
    for i in range(metvars):
        raveled = data[:, i, :, :].reshape(-1)
        m = raveled.mean()
        s = raveled.std()
        
        normalized[:, i, :, :] =  (data[:, i, :, :] - m)/s
    
    return normalized, means, storage

In [105]:
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization

In [117]:
x, means_x, storage_x = batch_normalize(train)

In [118]:
y, means_y, storage_y = batch_normalize(true)

In [83]:
# normalized, means, storage

## Normalize

In [84]:
units = normalized.shape

In [106]:
#cell = keras.layers.ConvLSTM2DCell(filters, kernel_size, strides=(1, 1), padding='same', data_format=None,
#                            dilation_rate=(1, 1), activation='tanh', recurrent_activation='hard_sigmoid', 
#                            use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
#                            bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None,
#                            recurrent_regularizer=None, bias_regularizer=None, 
#                            kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, 
#                            dropout=0.0, recurrent_dropout=0.0)

In [88]:
from keras.models import Sequential

In [102]:
x.shape, y.shape

((2184, 4, 35, 60), (2184, 1, 35, 60))

In [116]:
#x = x.reshape(-1, 2184, 4, 35, 60)
#y = y.reshape(-1, 2184, 1, 35, 60)

In [120]:
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization

#from keras.layers import keras.layers.

model =  Sequential()

# Begin with 2D convolutional LSTM layer
model.add(ConvLSTM2D(filters = 256, kernel_size = (3, 3),
                     input_shape = (None, 4, 35, 60),
                     padding = 'same', return_sequences=True)) 
"""
model.add(keras.layers.ConvLSTM2D(filters = 128, kernel_size = (3, 3),
                                  #input_shape = (256, 35, 60, 1),
                                  padding='same', return_sequences=True)) 

model.add(keras.layers.ConvLSTM2D(filters=64, kernel_size=(3, 3),
                                  #input_shape=(None, 35, 60, 1),
                                  padding='same', return_sequences=True)) 

# Add 3x hidden 2D convolutions LSTM layers    
# Begin with 2D convolutional LSTM layer
model.add(keras.layers.ConvLSTM2D(filters=1, kernel_size=(3, 3),
                                  #input_shape=(None, 40, 40, 1),
                                  padding='same', return_sequences=True)) """
# Prepare model for training
model.compile(
      loss = "mse",
      metrics = ["mae"],
      optimizer = "adam"
)

history = model.fit(x,
                    y,
                    batch_size = 32,
                    epochs = 40,
                    validation_split = 0.2,
                    )

plot(history)

ValueError: Error when checking input: expected conv_lst_m2d_22_input to have 5 dimensions, but got array with shape (2184, 4, 35, 60)

In [ ]:

# Add 3x hidden 2D convolutions LSTM layers
# Begin with 2D convolutional LSTM layer
model.add(keras.layers.ConvLSTM2DCell(filters = 128, kernel_size=(3,3), strides=(1, 1), padding='same', data_format=None,
                            dilation_rate=(1, 1), activation='tanh', recurrent_activation='hard_sigmoid', 
                            use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
                            bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None,
                            recurrent_regularizer=None, bias_regularizer=None, 
                            kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, 
                            dropout=0.0, recurrent_dropout=0.0) )

# Begin with 2D convolutional LSTM layer
model.add(keras.layers.ConvLSTM2D(filters = 64, kernel_size=(3,3), strides=(1, 1), padding='same', data_format=None,
                            dilation_rate=(1, 1), activation='tanh', recurrent_activation='hard_sigmoid', 
                            use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
                            bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None,
                            recurrent_regularizer=None, bias_regularizer=None, 
                            kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, 
                            dropout=0.0, recurrent_dropout=0.0) )
          